In [ ]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [ ]:
import src.pipeline.pre_processing as pre_processing
import src.pipeline.output_schemas as output_schemas
import time
import glob
import os

In [ ]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

In [ ]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [ ]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [ ]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [ ]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')

In [ ]:
#census.to_csv('output/pre-processing/census.csv')
census

## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [ ]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [ ]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

## KS2 and KS4 processing

In [ ]:
ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')

In [ ]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

In [ ]:
ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')

In [ ]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [ ]:
(trust_ar, academy_ar) = pre_processing.prepare_aar_data('data/academy_ar.xlsx')

In [ ]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

In [ ]:
#trust_ar.to_csv('output/pre-processing/trust_ar.csv')
trust_ar

Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [ ]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [ ]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [ ]:
academies = pre_processing.build_academy_data('data/academy_master_list.csv', 
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, trust_ar, ks2, ks4)

In [ ]:
academies.to_csv('output/pre-processing/academies.csv', columns=output_schemas.academies_output)
academies.sort_index()

Merge required census and cdc data to the maintained schools data set

In [ ]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [ ]:
maintained_schools_output = [
    "number of pupils whose first language is known or believed to be other than English",
    "Utilities_Water and sewerage:",
    "Utilities_Energy",
    "UrbanRural (name)",
    "UKPRN",
    "Type",
    "Total pupils",
    "Total School Workforce (Headcount)",
    "Total School Workforce (Full-Time Equivalent)",
    "Total Number of Teaching Assistants (Headcount)",
    "Total Number of Teaching Assistants (Full-Time Equivalent)",
    "Total Number of Teachers in the Leadership Group (Headcount)",
    "Total Number of Teachers in the Leadership Group (Full-time Equivalent)",
    "Total Number of Teachers (Headcount)",
    "Total Number of Teachers (Full-Time Equivalent)",
    "Total Number of Auxiliary Staff (Headcount)",
    "Total Number of Auxiliary Staff (Full-Time Equivalent)",
    "Total Internal Floor Area",
    "Total Income   I01 to I18",
    '"Total Income   I01 to I08, I11 to I15, I18"',
    "Total Expenditure  E01 to E32",
    '"Total Expenditure  E01 to E29 and E31 to E32 minus I9, I10, I16 and I17"',
    "TelephoneNum",
    "Teaching and Teaching support staff_Teaching staff",
    "Teaching and Teaching support staff_Supply teaching staff",
    "Teaching and Teaching support staff_Educational consultancy",
    "Teaching and Teaching support staff_Education support staff",
    "Teaching and Teaching support staff_Agency supply teaching staff",
    "Teachers with Qualified Teacher Status (%) (Headcount)",
    "Statutory Low Age",
    "Status",
    "SchoolWebsite",
    "SchoolPhaseType",
    "SchoolCapacity",
    "School Financial Position",
    "School Balance",
    "Revenue Reserve   B01 plus B02 plus B06",
    "Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers)",
    "Prov_VI",
    "Prov_SPLD",
    "Prov_SLD",
    "Prov_SLCN",
    "Prov_SEMH",
    "Prov_PMLD",
    "Prov_PD",
    "Prov_OTH",
    "Prov_MSI",
    "Prov_MLD",
    "Prov_HI",
    "Prov_ASD",
    "Progress8Measure",
    "Progress8Banding",
    "Primary Need VI",
    "Primary Need SPLD",
    "Primary Need SLD",
    "Primary Need SLCN",
    "Primary Need SEMH",
    "Primary Need PMLD",
    "Primary Need PD",
    "Primary Need OTH",
    "Primary Need MSI",
    "Primary Need MLD",
    "Primary Need HI",
    "Primary Need ASD",
    "Premises staff and services_Premises staff",
    "Premises staff and services_Other occupation costs",
    "Premises staff and services_Maintenance of premises",
    "Premises staff and services_Cleaning and caretaking",
    "Postcode",
    "Percentage SEN",
    "Percentage Free school meals",
    "PFI School",
    "Overall Phase",
    "Other grants and payments",
    "Other costs_Supply teacher insurance",
    "Other costs_Staff-related insurance",
    "Other costs_Staff development and training",
    "Other costs_Special facilities",
    "Other costs_Rent and rates",
    "Other costs_PFI charges",
    "Other costs_Other insurance premiums",
    "Other costs_Interest charges for loan and bank",
    "Other costs_Indirect employee expenses",
    "Other costs_Grounds maintenance",
    "Other costs_Direct revenue financing",
    "OfstedRating (name)",
    "OfstedLastInsp",
    "OfficialSixthForm (code)",
    "NurseryProvision (name)",
    "Number of Vacant Teacher Posts",
    "Non-educational support staff_Professional services (non-curriculum)",
    "Non-educational support staff_Other staff",
    "Non-educational support staff_Administrative and clerical staff",
    "No of pupils in 6th form",
    "No Teachers",
    "MSOA (code)",
    "Lowest age of pupils",
    "London Weighting",
    "Lead school in federation",
    "LastChangedDate",
    "LSOA (code)",
    "LA Establishment Number",
    "LA (name)",
    "LA (code)",
    "Ks2Progress",
    "In-year Balance   Total Income (I01 to I18) minus Total Expenditure (E01 to E32)",
    "IT_ICT learning resources",
    "I18  Additional grant for schools",
    "I17  Community focused school facilities income",
    "I16  Community focussed school funding and   or grants",
    "I15  Pupil focussed extended school funding and   or grants",
    "I13  Donations and or private funds",
    "I12  Income from contributions to visits etc",
    "I11  Receipts from other insurance claims",
    "I10  Receipts from supply teacher insurance claims",
    "I08  Income from facilities and services",
    "I06  Other government grants",
    "I05  Pupil Premium",
    "I04  Funding for minority ethnic pupils",
    "I03  SEN funding",
    "I02  Funding for 6th form students",
    "I01  Funds delegated by the LA",
    "Highest age of pupils",
    "HeadName",
    "Gender (name)",
    "GOR (name)",
    "FullTimeOtherHeadCount",
    "FullTimeOther",
    "FTE of Teaching Assistants",
    "FTE of Support Staff",
    "FTE of Admin Staff",
    "EstablishmentNumber",
    "EstablishmentName",
    "Educational supplies_Learning resources (not ICT equipment)",
    "Educational supplies_Examination fees",
    "EHC plan",
    "E32 Community focused school costs",
    "E31  Community focused school staff",
    "Catering_Income from catering",
    "Catering_Catering supplies",
    "Catering_Catering staff",
    "BoardingEstablishment (name)",
    "Boarders (name)",
    "AverageAttainment",
    "Age Average Score",
    "AdmissionsPolicy (name)",
    "AdmissionsPolicy (code)",
    "Administrative supplies_Administrative supplies (non educational)",
    "% of teachers with QTS",
    "% of pupils with EAL",
    "% of pupils who are Boarders",
    "% of pupils known to be eligible for free school meals (Performa",
    "% of pupils known to be eligible for and claiming free school me",
]

maintained_schools['Premises staff and services_Premises staff']
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv', columns=maintained_schools_output)
maintained_schools

## Federation Capture




In [ ]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools)

In [ ]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

In [ ]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')

In [ ]:
a = set(academies.columns.values)
m = set(maintained_schools.columns.values)

print(a-m)
